# 데이터 I/O 준비

In [ ]:
!pip install xlrd
!pip install openpyxl
!pip install pandas

In [1]:
from google.colab import drive

drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import os

# 구글 드라이브 상의 경로를 복사해서 아래 경로에 추가한다.
ROOT_PATH = '/content/gdrive/MyDrive/인간공학심리 연구실/Project/자유대화_샘플_220117'

# 본 셀의 결과로 페르소나의 이름이 담긴 리스트가 출력되면 올바르게 경로가 지정된 것이다.
PERSONAS = os.listdir(ROOT_PATH)
print(PERSONAS)

['잠보', '프로페서A', '씬스틸러 지은수', '침대러버', '카페사장', '프로관심러', '요리사', '사서', '꼬마도넛', '페럿', '제이드']


# Output Data 형식 정의

In [3]:
DATA_INFO_COLUMN = [
                    'doc_id',
                    'persona_name_original',
                    'user_name_original',
                    'user_id',
                    'total_turn',
                    'total_sent',
                    'total_minutes',
                    'pause_count',
                    'total_pause',
                    'pause_hour',
                    'age',
                    'gender',
                    'edu',
                    'income',
                    'open_chat_exp',
                    'friend_persona',
                    'friend_user',
                    'attract_persona',
                    'attract_user',
                    'connect_persona',
                    'connect_user',
                    'respect_persona',
                    'respect_user',
                    'fun_persona',
                    'fun_user',
                    'natural_persona',
                    'natural_user',
                    'blri_persona',
                    'blri_user',
                    'persona_text_all',
                    'user_text_all',
                    'persona_summary',
                    'user_summary'
                    ]

# 페르소나별 처리

DATA_INFO_DATA 리스트에 추가될 행을 대화 파일 하나당 처리한다

## Helper Function

In [4]:
def age_parser(answer):
    answer = answer.replace('만', '')
    answer = answer.replace('세', '')
    answer = answer.strip()
    if answer.isnumeric():
        return answer
    else:
        print(answer)
        raise Exception

In [5]:
def pause_parser(talk):
    total_pause = 0
    pause_hour = []

    for txt in talk[talk.is_pause == 1].text:
        hour = 0
        lines = txt.split("\n")

        for line in lines:
            try:
                if '시간이 경과했습니다' in line:
                    hour = int(line.split('시간')[0])
                elif '일이 경과했습니다' in line:
                    hour = int(line.split('일')[0]) * 24
                else:
                    continue                
            except:
                hour = 1 # "셜록홈즈는 어때 ~1시간이 경과했습니다."를 처리하기 위함
                # print(txt)
                # print(line)
                # print(" ")
            
            total_pause += hour
            pause_hour.append(hour)
    
    return total_pause, pause_hour

In [6]:
gender_dict = { '여성': 1, '남성': 2, '기타': 3}
edu_dict = {'초등학교 졸업' : 1, '중학교 졸업': 2, '고등학교 졸업': 3, '대학교 졸업': 4, '대학원 이상': 5}
income_dict = { '100만원 이하': 1, '100만원 초과 ~ 200만원 이하': 2, '200만원 초과 ~ 300만원 이하': 3, '300만원 초과 ~ 400만원 이하': 4, '400만원 초과': 5}
survey_dict = {'전혀 아니다': 1, '아니다': 2, '그렇다': 3, '매우 그렇다': 4}

In [7]:
def survey_parser(survey_actor, survey_talker):
    SURVEY_DATA = dict()
    
    SURVEY_DATA['friend_persona'] = survey_dict[survey_talker.iloc[19].answer]
    SURVEY_DATA['friend_user'] = survey_dict[survey_actor.iloc[0].answer]
    SURVEY_DATA['attract_persona'] = survey_dict[survey_talker.iloc[21].answer]
    SURVEY_DATA['attract_user'] = survey_dict[survey_actor.iloc[2].answer]
    SURVEY_DATA['connect_persona'] = survey_dict[survey_talker.iloc[23].answer]
    SURVEY_DATA['connect_user'] = survey_dict[survey_actor.iloc[4].answer]
    SURVEY_DATA['respect_persona'] = survey_dict[survey_talker.iloc[25].answer]
    SURVEY_DATA['respect_user'] = survey_dict[survey_actor.iloc[6].answer]
    SURVEY_DATA['fun_persona'] = survey_dict[survey_talker.iloc[27].answer]
    SURVEY_DATA['fun_user'] = survey_dict[survey_actor.iloc[8].answer]
    SURVEY_DATA['natural_persona'] = survey_dict[survey_talker.iloc[29].answer]
    SURVEY_DATA['natural_user'] = survey_dict[survey_actor.iloc[10].answer]

    return SURVEY_DATA

In [8]:
def blri_parser(survey, subject):
    idx = 13 if subject == "actor" else 32

    blri = 0
    for i in range(16):
        ans = survey_dict[survey.iloc[idx+i].answer]
        if i in [1, 4, 5, 7, 8, 10, 11, 14]:
            ans = 5 - ans
        blri += ans
    return blri

In [9]:
import re 

def aggregate_text(talk):
    txt = talk.groupby('person_type')['text'].apply('\n'.join).reset_index()

    user = txt[txt.person_type == 'user'].text.item()

    persona = txt[txt.person_type == 'persona'].text.item()
    
    persona = re.sub('\d{2}시간이 경과했습니다.' , "", persona)
    persona = re.sub('\d{2}시간이 경과했습니다' , "", persona)

    persona = re.sub('\d{1}시간이 경과했습니다.' , "", persona)
    persona = re.sub('\d{1}시간이 경과했습니다' , "", persona)

    persona = re.sub('\d{1}일이 경과했습니다.' , "", persona)
    persona = re.sub('\d{1}일이 경과했습니다' , "", persona)
    
    return persona, user

# Main Processing

In [12]:
import pandas as pd
import numpy as np
import csv
import re

DATA_INFO_DATA = []
for persona in PERSONAS:
    print(f'{persona} 작업 시작')

    PERSONA_PATH = os.path.join(ROOT_PATH, persona)

    items = sorted(os.listdir(PERSONA_PATH))
    TALKS = pd.read_excel(os.path.join(PERSONA_PATH, items[0]), None) # DeepNatural_freeTalk_persona.xlsx

    for talk_id in TALKS.keys():
        print('.', end='')

        DATA_INFO = dict()
        talk = pd.read_excel(os.path.join(PERSONA_PATH, items[0]), sheet_name = talk_id)

        doc_id = talk['doc_id'][0]
        

        survey_actor = pd.read_excel(os.path.join(PERSONA_PATH, items[1], doc_id + '.xlsx'))
        survey_talker = pd.read_excel(os.path.join(PERSONA_PATH, items[2], doc_id + '.xlsx'))

        DATA_INFO['doc_id'] = doc_id
        DATA_INFO['persona_name_original'] = talk[talk['person_type'] == 'persona']['person_name_original'].unique()[0]
        DATA_INFO['user_name_original'] = talk[talk['person_type'] == 'user']['person_name_original'].unique()[0]
        DATA_INFO['user_id'] = talk[talk['person_type'] == 'user']['person_id'].unique()[0]
        DATA_INFO['total_turn'] = talk['doc_id'][0].split('_')[-2]
        DATA_INFO['total_sent'] = talk['doc_id'][0].split('_')[-1]
        DATA_INFO['total_minutes'] = (talk.time[len(talk)-1] - talk.time[0]).days * 24 * 60 + divmod((talk.time[len(talk)-1] - talk.time[0]).seconds, 60)[0]
        DATA_INFO['pause_count'] = talk.is_pause.sum()
        DATA_INFO['total_pause'], DATA_INFO['pause_hour'] = pause_parser(talk)
        DATA_INFO['age'] = age_parser(survey_talker.iloc[14].answer)
        DATA_INFO['gender'] = gender_dict[survey_talker.iloc[15].answer]
        DATA_INFO['edu'] = edu_dict[survey_talker.iloc[16].answer]
        DATA_INFO['income'] = income_dict[survey_talker.iloc[17].answer]
        DATA_INFO['open_chat_exp'] = 1 if survey_talker.iloc[18].answer == "예" else 2
        DATA_INFO.update(survey_parser(survey_actor, survey_talker))
        
        try:  
            DATA_INFO['blri_persona'] = blri_parser(survey_actor, "actor")
        except KeyError:
            DATA_INFO['blri_persona'] = np.NaN
        
        try:
            DATA_INFO['blri_user'] = blri_parser(survey_talker, "talker")
        except KeyError:
            DATA_INFO['blri_user'] = np.NaN

        DATA_INFO['persona_text_all'], DATA_INFO['user_text_all'] = aggregate_text(talk)
        DATA_INFO['persona_summary'] = survey_actor.iloc[12].answer
        DATA_INFO['user_summary'] = survey_talker.iloc[31].answer

        DATA_INFO_DATA.append(DATA_INFO)
    print(f'\n{persona} 작업 완료')

f = open(os.path.join(ROOT_PATH, "data_info.csv"), "w")
writer = csv.DictWriter(f, fieldnames=DATA_INFO_COLUMN)
writer.writeheader()
writer.writerows(DATA_INFO_DATA)
f.close()    

잠보 작업 시작
........................................................................................................................................................................................................................................................................
잠보 작업 완료
프로페서A 작업 시작
.............................................................................................................................................................................................................................................
프로페서A 작업 완료
씬스틸러 지은수 작업 시작
........................
씬스틸러 지은수 작업 완료
침대러버 작업 시작
..........................................................................................................................................
침대러버 작업 완료
카페사장 작업 시작
..........................................................................................................................................
카페사장 작업 완료
프로관심러 작업 시작
.....